In [ ]:
%matplotlib inline
# import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os,sys
from PIL import Image
from helpers import *

## 1. Load data

In [ ]:
# Loaded a set of images
root_dir = "../dataset/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(20, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

imgs[0].shape, gt_imgs[0].shape

In [ ]:
# # Show first image and its groundtruth image
# cimg = concatenate_images(imgs[1], gt_imgs[1])
# fig1 = plt.figure(figsize=(20, 10))
# plt.imshow(cimg, cmap='Greys_r', vmin=0, vmax=1)

## 2. Generate the input and extract the outputs.
From each image extract the inputs (patches) and generate a list of features for each one of them.

In [ ]:
n = 20 # number of images
X = imgs_to_inputs(imgs[:n])
Y = imgs_to_outputs(gt_imgs[:n])

X.shape, Y.shape

Just check the **output** has been generated correctly.

In [ ]:
# Print feature statistics
Y0 = np.where(Y==0)[0]
Y1 = np.where(Y==1)[0]
print(str(len(Y1)) + ' inputs are classified as background (0)')
print(str(len(Y0)) + ' inputs are classified as road (1)')

## 3. Train a model

In [ ]:
print('Confusion matrix:')
confusion_matrix = pd.DataFrame(
    data = [['True Negative (TN)', 'False Negative (FN)'], ['False Positive (FP)', 'True Positive (TP)']],
    index = pd.MultiIndex(
        levels=[['predicted (Z)'], ['0', '1']],
        labels=[[0, 0], [0, 1]]),
    columns = pd.MultiIndex(
        levels=[['actual (Y)'], ['0 (background)', '1 (road)']],
        labels=[[0, 0], [0, 1]]),
)
display(confusion_matrix)
print('TN + FN + FP + TP = number of predictions')

- Logistic regression:

In [ ]:
from sklearn import linear_model

logreg = linear_model.LogisticRegression(C=1e5, class_weight="balanced")
# train
logreg.fit(X, Y)
# predict
Z = logreg.predict(X)

FNR = np.sum((Z == 0) & (Y == 1)) / float(len(Z))
TNR = np.sum((Z == 0) & (Y == 0)) / float(len(Z))
confusion_matrix.iloc[0, 0], confusion_matrix.iloc[0, 1] = TNR, FNR

FPR = np.sum((Z == 1) & (Y == 0)) / float(len(Z))
TPR = np.sum((Z == 1) & (Y == 1)) / float(len(Z))
confusion_matrix.iloc[1, 0], confusion_matrix.iloc[1, 1] = FPR, TPR

# print('True positive rate = ' + str(TPR) + ' (#correctly predicted road patches / #predicted road patches)')
print('Ratio of overall correct predictions: ' + str(np.sum(Z==Y)/float(len(Y))))
display(confusion_matrix)
print('Lot of non road patches are classified as road')

## 4. Validation

In [ ]:
validation_img = imgs[13]

Xi = img_to_inputs(validation_img)
Zi = logreg.predict(Xi)

In [ ]:
# Display prediction as an image
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 

w, h = 400, 400
predicted_im = label_to_img(w, h, patch_size, patch_size, Zi)
# cimg = concatenate_images(imgs[img_idx], predicted_im)
# plt.imshow(cimg, cmap='Greys_r', vmin=0, vmax=1)

new_img = make_img_overlay(validation_img, predicted_im)

plt.imshow(new_img)